In [47]:
import os
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [19]:
BASE_DIR = os.path.dirname(Path(os.path.abspath('')))
DATABASE_DIR = os.path.join(BASE_DIR, "db")
DATABASE_FILE_PATH = os.path.join(DATABASE_DIR, "game-data.sqlite")
engine = create_engine(f"sqlite:///{DATABASE_FILE_PATH}")

In [140]:
unblocked_query = """
WITH GoalieIdJoin AS (
SELECT
shots.gameId,
shots.season,
shots.TeamId,
shots.TeamName,
shots.isHomeTeam,
shots.Prev25FenwickForPerHour,
shots.Prev25FenwickAgainstPerHour,
pp.Prev25ShotsFor5v4PerHour,
pp.Prev25ShotsAgainst4v5PerHour,
spct.Prev25ShootingPercentage,
playerId as GoalieId
FROM UnblockedShotGenSupLast25 AS shots 
LEFT JOIN goalie_saves as gs ON shots.gameId = gs.gameId AND ((shots.isHomeTeam = 1 AND gs.homeRoad = 'H') OR (shots.isHomeTeam = 0 AND gs.homeRoad = 'R')) and gs.gamesStarted = 1 
LEFT JOIN ShotGenSup5v4Last25 as pp ON pp.gameId = shots.gameId AND pp.teamId = shots.TeamId
LEFT JOIN TeamScoringPercentage as spct ON spct.gameId = shots.gameId AND spct.eventOwnerTeamId = shots.TeamId
)
SELECT GoalieIdJoin.*,
gsave.diluted_save_pct,
games.homeScore > games.visitingScore AS HomeTeamWin
FROM GoalieIdJoin
LEFT JOIN GoalieSavePct as gsave ON gsave.goalieId = GoalieIdJoin.GoalieId
LEFT JOIN games ON games.id = GoalieIdJoin.gameId
"""
UnblockedShotGenSup = pd.read_sql_query(unblocked_query, engine)

In [103]:
df = UnblockedShotGenSup

In [104]:
df_nn = df[df["Prev25FenwickAgainstPerHour"].notna()]

In [114]:
df_home = df_nn[df_nn["isHomeTeam"] == 1].rename(columns={
    "TeamName": "HomeTeamName",
    "TeamId": "HomeTeamId",
    "Prev25FenwickForPerHour": "HomePrev25FenwickForPerHour",
    "Prev25FenwickAgainstPerHour": "HomePrev25FenwickAgainstPerHour",
    "Prev25ShotsFor5v4PerHour": "HomePrev25ShotsFor5v4PerHour",
    "Prev25ShotsAgainst4v5PerHour": "HomePrev25ShotsAgainst4v5PerHour",
    "Prev25ShootingPercentage": "HomePrev25ShootingPercentage",
    "GoalieId": "HomeGoalieId",
    "diluted_save_pct": "HomeDilutedSavePct"
}).drop(["isHomeTeam"], axis=1)
df_away = df_nn[df_nn["isHomeTeam"] == 0].rename(columns={
    "TeamName": "AwayTeamName",
    "TeamId": "AwayTeamId",
    "Prev25FenwickForPerHour": "AwayPrev25FenwickForPerHour",
    "Prev25FenwickAgainstPerHour": "AwayPrev25FenwickAgainstPerHour",
    "Prev25ShotsFor5v4PerHour": "AwayPrev25ShotsFor5v4PerHour",
    "Prev25ShotsAgainst4v5PerHour": "AwayPrev25ShotsAgainst4v5PerHour",
    "Prev25ShootingPercentage": "AwayPrev25ShootingPercentage",
    "GoalieId": "AwayGoalieId",
    "diluted_save_pct": "AwayDilutedSavePct"
}).drop(["season", "isHomeTeam", "HomeTeamWin"], axis=1)

In [115]:
df_com = pd.merge(left=df_home, right=df_away, how="left", on="gameId")

In [135]:
df_std = df_com.copy()

In [138]:
cols_to_standardize = ["HomePrev25FenwickForPerHour", "HomePrev25FenwickAgainstPerHour", "HomePrev25ShotsFor5v4PerHour", "HomePrev25ShotsAgainst4v5PerHour", "HomePrev25ShootingPercentage", "HomeDilutedSavePct", "AwayPrev25FenwickForPerHour", "AwayPrev25FenwickAgainstPerHour", "AwayPrev25ShotsFor5v4PerHour", "AwayPrev25ShotsAgainst4v5PerHour", "AwayPrev25ShootingPercentage", "AwayDilutedSavePct"]
df_std[cols_to_standardize] = df_std.groupby('season')[cols_to_standardize].transform(
    lambda x: (x - x.mean()) / x.std()
)

In [139]:
df_std.head(20)

,gameId,season,HomeTeamId,HomeTeamName,HomePrev25FenwickForPerHour,HomePrev25FenwickAgainstPerHour,HomePrev25ShotsFor5v4PerHour,HomePrev25ShotsAgainst4v5PerHour,HomePrev25ShootingPercentage,HomeGoalieId,...,HomeTeamWin,AwayTeamId,AwayTeamName,AwayPrev25FenwickForPerHour,AwayPrev25FenwickAgainstPerHour,AwayPrev25ShotsFor5v4PerHour,AwayPrev25ShotsAgainst4v5PerHour,AwayPrev25ShootingPercentage,AwayGoalieId,AwayDilutedSavePct
0,2010020027,20102011,1.0,New Jersey Devils,-0.621922,0.096564,-1.119451,-3.041476,-1.033767,8455710,...,0,5.0,Pittsburgh Penguins,1.865888,-0.365052,-1.689887,-0.758799,-2.516405,8462161.0,-0.715372
1,2010020048,20102011,1.0,New Jersey Devils,-0.296603,-0.094101,-0.200081,-1.946146,-2.320455,8455710,...,0,21.0,Colorado Avalanche,0.396252,0.249678,-2.357457,1.378747,0.268350,8467950.0,-0.188681
2,2010020056,20102011,1.0,New Jersey Devils,-0.003958,-0.110100,0.572126,-2.139655,-2.377289,8455710,...,0,6.0,Boston Bruins,3.151845,1.036010,0.928432,1.249424,-0.726889,8460703.0,1.579586
3,2010020100,20102011,1.0,New Jersey Devils,-0.400088,0.072292,0.082281,-1.534918,-1.554693,8460704,...,0,7.0,Buffalo Sabres,0.794302,-0.893872,0.410526,0.671479,-0.403079,8468011.0,0.654865
4,2010020179,20102011,1.0,New Jersey Devils,-0.268979,-0.032316,-0.406049,-0.759377,-1.516934,8460704,...,0,3.0,New York Rangers,0.314161,0.324722,0.013064,0.613315,0.634415,8468685.0,1.370794
5,2010020211,20102011,1.0,New Jersey Devils,-0.213910,-0.233567,-0.439769,-1.038887,-1.755750,8460704,...,0,7.0,Buffalo Sabres,0.708916,-0.087558,-0.121792,-0.223804,-0.541564,8473523.0,0.256164
6,2010020227,20102011,1.0,New Jersey Devils,-0.287315,-0.302146,-0.458077,-0.771850,-1.489411,8455710,...,1,22.0,Edmonton Oilers,-1.540744,1.668079,-1.293354,2.293826,0.193982,8471227.0,0.653409
7,2010020299,20102011,1.0,New Jersey Devils,0.157239,-0.761975,-0.467758,-0.877912,-1.624193,8460704,...,1,15.0,Washington Capitals,-0.057713,-0.666507,0.188464,-0.219738,0.549032,8474651.0,0.753578
8,2010020313,20102011,1.0,New Jersey Devils,-0.045205,-0.835047,-0.571229,-0.989139,-1.383874,8460704,...,1,20.0,Calgary Flames,0.776971,-0.231320,0.385022,-0.751526,0.157768,8475431.0,-0.466973
9,2010020334,20102011,1.0,New Jersey Devils,-0.228808,-1.076437,-0.542523,-1.037411,-1.491907,8460704,...,1,4.0,Philadelphia Flyers,1.353017,0.069767,0.047334,-0.407347,1.421563,8462052.0,-0.503643
